<a href="https://colab.research.google.com/github/raihanewubd/24FALL-AI/blob/main/AI_Lab_4_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementation of Artificila Neural Network (ANN)**
Implemented by Al Hossain
---



> Diabetes Dataset Link: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm

### Load Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANN/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
class_counts = df['Outcome'].value_counts()
print(class_counts)

In [ ]:
df.describe()

In [ ]:
print("Null values in each column:")
print(df.isnull().sum())

In [ ]:
df_cleaned = df.dropna()
print("Null values after cleaning:")
print(df_cleaned.isnull().sum())

### Split the Dataset into Features and Labels


*   **Features:** (Independent variables) These are the input characteristics or attributes of the data points.
*   **Labels:** (Dependent Variables) These are the outcomes or target variables that we want to predict.



In [ ]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

In [ ]:
print(X.shape)
print(y.shape)

### Split the Dataset into Training and Testing Sets

*   **Training Set:** Used to teach the model the underlying patterns and relationships within the data.
*   **Test Set:** Evaluates the model's performance on data it hasn't seen before.



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
sizes = [len(X_train), len(X_test)]
labels = ['Training Set', 'Test Set']

plt.figure(figsize=(5, 5))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Training and Test Data')
plt.show()

### ANN Model Design and Implementation

In [ ]:
# Convert to tensor object
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class ANN_model(nn.Module):
    # For ANN we will require input layer, hidden layer and output layer
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features=2):
        # Inherit parent class i.e.nn.Module
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1, hidden2)
        self.out=nn.Linear(hidden2,out_features)
    # x will track the gradient descent and all the propogation
    def forward(self,x):
        # F we have all the activation functions
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [ ]:
model = ANN_model()

In [ ]:
print(model.parameters)

### Model Training

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
def train_model(model, X_train, y_train, epochs=10):
    loss_values = []

    for epoch in tqdm(range(epochs), desc='Training Epochs'):
        model.train()

        # Forward pass
        y_pred = model(X_train)

        # Compute loss
        loss = loss_function(y_pred, y_train)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_values.append(loss.item())

        if (epoch + 1) == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

    return loss_values

loss_values = train_model(model, X_train, y_train, epochs=1000)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(loss_values) + 1), loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.show()

### Model Evaluation

In [ ]:
def evaluate_model_with_class_accuracy(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)

        # Get predicted class labels
        _, predicted = torch.max(y_pred, 1)

        # Calculate overall accuracy
        accuracy = accuracy_score(y_test.numpy(), predicted.numpy())
        print(f'Overall Accuracy: {accuracy:.4f}')

        # Generate and print classification report
        report = classification_report(y_test.numpy(), predicted.numpy(), target_names=class_names)
        print("\nClassification Report:\n", report)

        # Calculate per-class accuracy
        for i, class_name in enumerate(class_names):
            class_accuracy = accuracy_score(y_test.numpy()[y_test.numpy() == i], predicted.numpy()[y_test.numpy() == i])
            print(f'Accuracy for {class_name}: {class_accuracy:.4f}')

class_names = ['0', '1']
evaluate_model_with_class_accuracy(model, X_test, y_test)

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

y_pred = model(X_test).argmax(dim=1).numpy()
plot_confusion_matrix(y_test.numpy(), y_pred, class_names)